# python 多线程问题
## 列表， 字典是线程安全的
## dataframe 不是线程安全的
### sleep 0.1 秒 0.01秒 OK 再小会卡住
### 线程数大概1E6会非常吃内存 2G, 危险

In [60]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import pandas as pd




def dosomthing(i):
    time.sleep(3)
    #print(i, " is ok")
  
    #######
    
    return i
    
with ThreadPoolExecutor(max_workers= int(1E5) ) as t:
    obj_list = []
    begin = time.time()
    for i in range(0, int(1E5) ):
        obj = t.submit(dosomthing, i)
        obj_list.append(obj)
    
    for future in as_completed(obj_list):
        data = future.result()
        # print(data)
    times = time.time() - begin
    print("多线程占据的全部时间"+str(times))
 
 ################# 


多线程占据的全部时间12.603969812393188


In [44]:
# 测试线程安全

In [62]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import pandas as pd

# 列表是线程安全的
thread_list = []
# dataframe不是线程安全的
my_df = pd.DataFrame([],index=[],columns=['text','label'])

# 
map = {}
text_map = {}
label_map = {}
def dosomthing(i):
    time.sleep(4)
    # print(i, " is ok")
    thread_list.append(str(i)+"               ok")
    text_map[i] = "ok" + str(i)
    label_map[i] = i
    
    return i
    
with ThreadPoolExecutor(max_workers= int(1E4)) as t:
    obj_list = []
    begin = time.time()
    for i in range(0, int(1E4)) :
        obj = t.submit(dosomthing, i)
        obj_list.append(obj)
    
    for future in as_completed(obj_list):
        data = future.result()
        # print(data)
    times = time.time() - begin
    print("多线程占据的全部时间"+str(times))
    print(len(thread_list))
    print(len(map))

map['text'] = text_map
map['label'] = label_map

frame3 = pd.DataFrame(map)
frame3

多线程占据的全部时间6.299389362335205
10000
0


,text,label
21,ok21,21
22,ok22,22
20,ok20,20
18,ok18,18
24,ok24,24
...,...,...
9981,ok9981,9981
9974,ok9974,9974
9973,ok9973,9973
9902,ok9902,9902


In [94]:
import thulac
import os
import re
import pandas as pd
import jieba

from gensim.models import Word2Vec
# 2. 读取停词表
def load_stop_word():
    stop_word = ['的', '。', '是', ' ', '\n', '日', '月', '.', '%','\u3000']
    stop_word_file = 'c:/data/stopwords'
    stop_word_dirs = os.listdir(stop_word_file)
    for textName in stop_word_dirs:
        filename = stop_word_file + '/' + textName
        with open(filename, 'r', encoding='utf-8') as file_object:
            line = file_object.readline()
            while line:
                stop_word.append(line[0:-1])
                line = file_object.readline()
    return stop_word

stop_word = load_stop_word()
print(len(stop_word))
print(stop_word[0:100])


# 2.5 分词前修改thulac源码中的bug

from contextlib import contextmanager
from io import StringIO
import time


@contextmanager
def add_clock_method_to_time():
    py_gt_3_8 = not hasattr(time, "clock")
    if py_gt_3_8:
        setattr(time, "clock", time.perf_counter)
    try:
        yield
    finally:
        if py_gt_3_8:
            delattr(time, "clock")
            
#分词
 # 默认模式，分词的同时进行词性标注
# thu = thulac.thulac(filt=True, seg_only=True)
word_type = {}
def cut_doc(doc):
     # 默认模式，分词的同时进行词性标注
    
    words = jieba.cut(doc)
    words_in_doc = ""
    for word in words:
        # print(word)
        if word in stop_word: continue
        words_in_doc +=  word + " "
        # word_type[word] = wordType
        
    return words_in_doc

words_in_doc = cut_doc("用来控制训练的文章数量")
words_in_doc

3908
['的', '。', '是', ' ', '\n', '日', '月', '.', '%', '\u3000', '--', '?', '“', '”', '》', '－－', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', "a's", 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'came', 'can', 'cannot', 'cant', "can't", 'cause', 'causes', 'certain']


'控制 训练 文章 数量 '

# 多线程读取原生 的 数据集2G  每个类别读3000个新闻


In [100]:

doc_map = {}
label_map = {}
map = {}
DOC_COUNT_MAX = 2000
wrong_num = []

def read_file_cut(doc_file_name, doc_id, label_index):
    with open(doc_file_name, 'r', encoding='utf-8') as file_object:
        doc_content = ""
        line = file_object.readline()
        while line:
            doc_content += line
            line = file_object.readline()
        wrong = 1
        for _ in range(0,4):
            try:
                doc_map[doc_id] = cut_doc(doc_content)
                wrong = 0
                break
            except Exception as e:
                continue
        if wrong == 1:
            wrong_num.append(doc_id)
        label_map[doc_id] = int(label_index)
    return 0

my_df = pd.DataFrame([],index=[],columns=['text','label'])

label_num = {}

with ThreadPoolExecutor(max_workers= int(10000)) as t:
    obj_list = []
    begin = time.time()
    
    news_file = 'c:/data/THUCNews'
    news_file_dirs = os.listdir(news_file)
    # print(news_file_dirs)
    
    label_index = -1
    for label in news_file_dirs:
        label_index += 1
        label_num[label] = label_index
        
        file = news_file + '/' + label
        doc_ids = os.listdir(file)
        doc_num = 0
        for doc_name in doc_ids:
            doc_num += 1
            if doc_num >= DOC_COUNT_MAX:
                break
                
            doc_file_name = file + '/' + doc_name
            doc_id = doc_name[0:-4]
            doc_content = ""
            
            
            obj = t.submit(read_file_cut, doc_file_name, doc_id, label_index)
            obj_list.append(obj)    
            
      
    for future in as_completed(obj_list):
        data = future.result()
    times = time.time() - begin
    print("多线程占据的全部时间"+str(times))
    print(len(obj_list))
    print(len(doc_map))
    print(len(label_map))
    
map['text'] = doc_map
map['label'] = label_map


my_df = pd.DataFrame(map)
train_df = my_df
print("错误的次数", len(wrong_num) )
print(train_df.shape)
train_df.to_csv("c:/data/news/train_data1.csv")


多线程占据的全部时间503.9964108467102
27986
27986
27986
错误的次数 0
(27986, 2)


In [113]:
# Count Vectors + RidgeClassifier
print(train_df)
print(train_df['label'])
train_df['label'] = train_df['label'].astype('int')
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = train_df.sample(frac = 1) #打乱样本

all_data_size = 1200
train_data_size = 1000
test_data_size = all_data_size - train_data_size


vectorizer = CountVectorizer(max_features=3000)
train_test = vectorizer.fit_transform(train_df['text'])

print(train_test[0].shape )
clf = RidgeClassifier()
clf.fit(train_test[:train_data_size], train_df['label'].values[:train_data_size])
val_pred = clf.predict(train_test[train_data_size:])
print(f1_score(train_df['label'].values[train_data_size:], val_pred, average='macro'))
# 0.28


                                                     text  label
0       马晓旭 意外 受伤 国奥 警惕 无奈 大雨 青睐 殷家 军 记者 傅亚雨 沈阳 报道 来到 ...      0
1       商瑞华 首战 复仇 心切 中国 玫瑰 美国 方式 攻克 瑞典 多曼来 瑞典 商瑞华 首战 求...      0
100     辽足 签约 危机 引 注册 难关 高层 威逼利诱 合同 笑里藏刀 新浪 体育讯 24 辽足 ...      0
10000   阿的江 八一 需 定位 机会 进 没进 新浪 体育讯 12 24 回到 主场 北京 金隅 迎...      0
1000    揭秘 谢亚龙 带走 总局 电话 骗局 复制 南杨 轨迹 体坛周报 特约记者 张锐 北京 报道...      0
...                                                   ...    ...
800787  全球 藏家 吹 印度 风 中国 知名 藏家 张锐 杨斌 出手 印度 当代艺术 走进 中国 理...     13
800746  贵 国庆 邮票 已翻 万倍 贵金属 藏品 交易 火爆 邮票 最贵 翻 万倍 最平刚 面值 记...     13
800884  还原 购物 天堂 迪拜 真实 面貌 离天 建筑 鸟瞰 迪拜 迪拜 阿拉伯 世界 一颗 耀眼 ...     13
800612  傅抱石 艺术 作品 行情 走向 步入 21 世纪 傅抱石 作品 异军突起 捷报频传 一本 毛...     13
800835  烟雾 袅袅 圈子 雪茄 赏 最低 调 奢华 策划 编辑 宋慧敏     文 紫檀      ...     13

[27986 rows x 2 columns]
0          0
1          0
100        0
10000      0
1000       0
          ..
800787    13
800746    13
800884    13
800612    13
800835    13
Name: label, Length: 27986, dtype: int32
(1, 3000)


In [114]:
# TF-IDF +  RidgeClassifier

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score


tfidf = TfidfVectorizer(ngram_range=(1,4), max_features=300)

train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:train_data_size], train_df['label'].values[:train_data_size])

val_pred = clf.predict(train_test[train_data_size:])

print(f1_score(train_df['label'].values[train_data_size:], val_pred, average='macro'))

0.8033556405276042


In [116]:
import numpy as np
useless = set()

doc_data = train_df['text']
common_text = []
for i in range(0, doc_data.size):
    doc = str(doc_data[i]).split(" ")
    common_text.append(doc)

# 词向量的维度，样本够的话300-500。
# min_count 最小次数计数，出现次数低于这个数的就不要了
# workers  线程数
# window 窗口大小
# sample 负例采样设置aa


vector_size = 500
model = Word2Vec(sentences=common_text, vector_size=vector_size, window=5, min_count=4,
                  workers=12, sample=1e-5)
# model.save("word2vec.model")
vector = model.wv['中国']

# 词向量 合成 文本向量
doc_v_list = np.array([])

for split_doc in common_text:
    used_word = float(0.0)
    vector = vector * 0
    for word in split_doc:

        used_word += 1.0
        try:
            vector += model.wv[word]
        except Exception as e:
            # print(e.__traceback__.tb_frame.f_globals["__file__"])  # 发生异常所在的文件
            # print(e.__traceback__.tb_lineno)  # 发生异常所在的行数
            used_word -= 1.0

    vector /= used_word /200
    doc_v_list = np.append(doc_v_list, vector)

train_test = doc_v_list.reshape(len(common_text), vector_size)

# 使用word2vec 
clf = RidgeClassifier()
clf.fit(train_test[:train_data_size], train_df['label'].values[:train_data_size])

val_pred = clf.predict(train_test[train_data_size:])
print(f1_score(train_df['label'].values[train_data_size:], val_pred, average='macro'))

0.9137776175599605
